## SQLite basic operations / SQLite podstawowe operacje

In this notebook we are going to do basic SQLite operations like creating table, inserting, updating and deleting data inside a table, fetching and displaying data with ordering and basic pattern matching.
We are going to manually add a few Starcraft 2 units.

Below is a basic table of units to be put inside database table.

W tym zeszycie zaprezentuję podstawowe operacje SQLite jak tworzenie tabeli, umieszczanie, aktualizowanie i usuwanie danych z tabeli oraz szukawnie, sortowanie i pokazanie rekordów na ekranie.

Poniżej jest tabela z jednostami to umieszczenia w tabeli.


| Name          | Mineral | Damage | Hit Points | Range |
|---------------|---------|--------|------------|-------|
| Marine        | 50      | 6      | 45         | 5     |
| Siege Tank    | 150     | 15     | 175        | 7     |
| Battlecruiser | 4000    | 8      | 550        | 6     |

In [1]:
import sqlite3

In [2]:
# Creating SQLite connection
# Tworzenie połączenia z bazą danych

conn = sqlite3.connect('data/sc2_basic_units.db')

In [3]:
# Creating a cursor
# Tworzenie cursora. Połączenie i cursor są potrzebne do wykonywania poleceń na bazie danych

c = conn.cursor()

In [ ]:
# Deleting a table if it happens to exist
# Usówanie tabeli jeśli ona już istnieje



In [4]:
# Creating a table terran_units to store terran_units
# Tworzymy tablelę terran_units do przechowywania jednostek terrana

c.execute("""CREATE TABLE terran_units (
        name text,
        mineral integer,
        damage real,
        hit_points integer,
        range text
    )""")

OperationalError: table terran_units already exists

In [5]:
# Inserting one record into table terran units
# Umieszczenie jednego rekorku jednostki w tabeli terran_units

c.execute("INSERT INTO terran_units VALUES('Marine', 50, 6, 45, 5)")

#### Insert many records into table / Umieszczanie wielu rekordów w tabeli
First we create a list of tuples, with each tuple corresponding to the record structure inside out table.
Later we use executemany function using ? placeholders which inserts all the records from our list.

Najpierw tworzymy listę krotek, gdzie każda krotka ma strukturę rekordu w tabeli.
Potem używamy funkcji executemany(), która wykonuje zawołanie SQL dla każdego elementu listy.

In [6]:
units = [('Marauder', 100, 10, 125, 10), ('Siege Tank', 150, 15, 175, 7), ('Battlecruiser', 400, 8, 550, 6)]

In [7]:
# ? - placeholders
# ? - zamiennik

c.executemany("INSERT INTO terran_units VALUES (?,?,?,?,?)", units)

In [8]:
# Commiting changes
# Zapisywanie zmian

conn.commit()

In [9]:
# Using SELECT clause to fetch data to verify that the data is in the database
# Używanie SELECT, żeby pobrać dane z bazy danych

c.execute("SELECT * FROM terran_units")
items = c.fetchall()

In [10]:
for i in items:
    print(f"{i[0]} {i[1]} {i[2]} {i[3]} {i[4]}")

Marine 50 6.0 45 5
Marauder 100 10.0 125 10
Siege Tank 150 15.0 175 7
Battlecruiser 400 8.0 550 6


#### Fetching and printing PRIMARY KEY / Zbieranie

PRIMARY KEY is created by sqlite by default even if it's not explicitly asked for.
PRIMARY KEY (KLUCZ GŁÓWNY) jest domyślnie stworzony przez bazę danych SQLite.

In [11]:
c.execute("SELECT rowid, name FROM terran_units")
id_name = c.fetchall()

In [12]:
for i in id_name:
    print(f"{i[0]} {i[1]}")

1 Marine
2 Marauder
3 Siege Tank
4 Battlecruiser


In [13]:
# Finding units which have HP higher than 120 using the WHERE clause
# Znajdowanie jednostek które mają więcej niż 120 HP używając klauzuli WHERE

c.execute("SELECT * FROM terran_units WHERE hit_points > 120")

In [14]:
items = c.fetchall()
for i in items:
    print(f"{i[0]} {i[1]} {i[2]}")

Marauder 100 10.0
Siege Tank 150 15.0
Battlecruiser 400 8.0


In [15]:
# Finding units which have M starting with the name of the unit
# Znajdowanie jednostek, których nazwa zaczyna się od 'M'

c.execute("SELECT * FROM terran_units WHERE name LIKE 'M%'")

In [16]:
items = c.fetchall()
for i in items:
    print(f"{i[0]} {i[1]} {i[2]}")

Marine 50 6.0
Marauder 100 10.0


In [17]:
# Updating records - Marines are to cheap, we want to increase its cost to 75 minerals
# Aktualizowanie rekoru - Marines są zbyt tanie, zwiększymy ich koszt do 75 minerałów

c.execute("""UPDATE terran_units SET mineral = 75 WHERE name = 'Marine'""")

In [18]:
c.execute("SELECT * FROM terran_units")
items = c.fetchall()
for i in items:
    print(f"{i[0]} {i[1]} {i[2]} {i[3]} {i[4]}")

Marine 50 6.0 45 5
Marauder 100 10.0 125 10
Siege Tank 150 15.0 175 7
Battlecruiser 400 8.0 550 6


In [19]:
# Updating records by rowid
# Aktualizowanie rekordu używając rowid

c.execute("""UPDATE terran_units SET damage = 14 WHERE rowid = 3""")

In [20]:
# Deleting records - removing battlecruiser from the database (droping)
# Usuwanie rekordu - Battlecruiser jest za silną jednostką więc go usuwamy

c.execute("DELETE FROM terran_units WHERE rowid = 4")

In [21]:
c.execute("SELECT * FROM terran_units")
items = c.fetchall()
for i in items:
    print(f"{i[0]} {i[1]} {i[2]} {i[3]} {i[4]}")

Marine 50 6.0 45 5
Marauder 100 10.0 125 10
Siege Tank 150 14.0 175 7


#### Ordering - ORDER BY keyword / Sortowanie rekordów
ASC ascending
DESC descending
ASC - dane sortujemy rosnąco
DESC - dane sortujemy malejąco

In [22]:
c.execute("SELECT * FROM terran_units ORDER BY damage DESC")

In [23]:
items = c.fetchall()
for i in items:
    print(f"{i[0]} {i[1]} {i[2]} {i[3]} {i[4]}")

Siege Tank 150 14.0 175 7
Marauder 100 10.0 125 10
Marine 50 6.0 45 5


In [24]:
# Limiting results - LIMIT KEYWORD
# Sorting units by highest high points limiting results to two

# Ograniczenie wyników do 2 z sortowaniem od najwyższego wyniku

c.execute("SELECT * FROM terran_units ORDER BY hit_points DESC LIMIT 2")
items_id = c.fetchall()

for i in items_id:
    print(f"{i[0]} {i[1]} {i[2]} {i[3]}")

Siege Tank 150 14.0 175
Marauder 100 10.0 125


In [25]:
# Droping table - deletes a table
# Usuwanie tabeli

c.execute("DROP TABLE terran_units")

In [26]:
# Closing connection
# Zamykanie połączenia
conn.close()